In [1]:
# Automation in Construction
idNo = '271427'

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv

In [3]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [4]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['BIM']
# searchTerm2 = ['Sustainability']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "%20"

    fileIndex = text2.rfind("%20")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "%20"

    fileIndex = text1.rfind("%20")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green%20Building', 'Green%20Buildings', 'Sustainable%20building', 'Sustainable%20buildings', 'Environmentally%20friendly%20design', 'Sustainability', 'Sustainable%20development', 'Energy', 'Water', 'Waste', 'Thermal%20comfort', 'Electricity', 'Solar', 'Wind%20power', 'Green%20infrastructure', 'Life%20cycle%20assessment']
['BIM', 'Building%20Information%20Modelling', 'Building%20information%20model', '6D%20BIM', 'Digital%20construction', 'Smart%20construction', 'Digital%20engineering']


In [6]:
def getData(link):
    browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
    browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('span',{'class':'title-text'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title
    
    absResults = parsed_html.find('div',{'class':'abstract author'})
    
    if absResults is not None:
        try:            
            df.loc[row,'section0'] = absResults.find('h2').text
            df.loc[row,'sectionData0'] = absResults.find('div').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

        
    sections = parsed_html.find('div',{'class':'Body u-font-serif'})
    if sections is not None:        
        sections = sections.find_next('div')
        count = 1 
        for section in sections:
            if section is not None: 
                sectionNo = 'section' + str(count)
                sectionData = 'sectionData' + str(count)
                try:
                    df.loc[row,sectionNo] = section.find('h2',{'class':'u-h3 u-margin-l-top u-margin-xs-bottom'}).text
                except:
                    df.loc[row,sectionNo] = "Some issue"
                    
                data = ''
                for para in section.findAll('p'):
                    data = data + para.text
                df.loc[row,sectionData] = data
            else:
                df.loc[row,sectionNo] = "None"
            count = count + 1        

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [8]:
baseUrl = 'https://www.sciencedirect.com/search/api?tak=%22'

querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = 'https://www.sciencedirect.com/search?qs=%22' + item1 + '%22%20%2B%20%22' + item2 + '%22'
        print("base query: ",sq)
        query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
        
        # Call webdriver         
        browser = webdriver.Safari()
        browser.get(query)
        respData = browser.page_source
        browser.quit()

        # Clear html         
        respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
        respData = respData.replace('</pre></body></html>',"")
        
        py_dic = json.loads(respData)
        
        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'resultsFound':
                pageResults = py_dic[key]
        print("pageResults: ",pageResults)                
                                
        if pageResults > 25:
            page = pageResults // 25
            lastPage = pageResults % 25
            if lastPage > 0:
                page = page + 1
        else:
            page = 1
        print("page: ",page)

        #         for p in range(2):        
        for p in range(page):
            print("In page: ",p)
            if p == 0:
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
            else:
                offset = '&offset='+str(p*25)
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
                
            print("search query: ",query)  
                
            # Call webdriver         
            browser = webdriver.Safari()
            browser.get(query)
            respData = browser.page_source
            browser.quit()
            # Load Json Data 

            respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
            respData = respData.replace('</pre></body></html>',"")

            py_dic = json.loads(respData)                
            # Quit the browser 
            links = []
            for key in py_dic:
                if key == 'searchResults':
                    for searchR in py_dic[key]:
                        for search in searchR:
                            if search == 'pdf':
                                for linkR in searchR[search]:
                                    if linkR == 'getAccessLink':
                                        baseLink = 'https://www.sciencedirect.com/'
                                        link = baseLink +searchR[search][linkR]
                                        print('Reading result: ',row)
                                        df.loc[row,'searchTerm1'] = item1
                                        df.loc[row,'searchTerm2'] = item2
                                        getData(link)                                    
                                        row = row + 1        

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Building%22
pageResults:  10
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Building%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  0
Title:  A performance data integrated BIM framework for building life-cycle energy efficiency and environmental optimization design
Reading result:  1
Title:  Building Information Modeling (BIM) for green buildings: A critical review and future directions
Reading result:  2
Title:  A BIM-WMS integrated decision support tool for supply chain management in construction
Reading result:  3
Title:  Integrating web map service and building information modeling for location and transportation analysis i

Title:  BIM-based information system for econo-enviro-friendly end-of-life disposal of construction and demolition waste
Reading result:  46
Title:  Implementing ‘BEAM Plus’ for BIM-based sustainability analysis
Reading result:  47
Title:  Enhancing environmental sustainability over building life cycles through green BIM: A review
Reading result:  48
Title:  Integration of LCA and LCC analysis within a BIM-based environment
Reading result:  49
Title:  How to tell if a BIM project is successful: A goal-driven approach
Reading result:  50
Title:  Mapping the knowledge domains of Building Information Modeling (BIM): A bibliometric approach
Reading result:  51
Title:  Green building assessment tool (GBAT) for integrated BIM-based design decisions
Reading result:  52
Title:  Publishing authoritative geospatial data to support interlinking of building information models
Reading result:  53
Title:  Building Information Modeling (BIM) for green buildings: A critical review and future direction

Title:  Interoperability specification development for integrated BIM use in performance based design
Reading result:  104
Title:  Implementing ‘BEAM Plus’ for BIM-based sustainability analysis
Reading result:  105
Title:  BIM IFC information mapping to building energy analysis (BEA) model with manually extended material information
Reading result:  106
Title:  A framework for the utilization of Building Management System data in building information models for building design and operation
In page:  1
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Energy%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  107
Title:  A performance data integrated BIM framework for building life-cycle energy efficiency and environmental optimization design
R

pageResults:  20
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Waste%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  159
Title:  Automated reinforcement trim waste optimization in RC frame structures using building information modeling and mixed-integer linear programming
Reading result:  160
Title:  BIM-based information system for econo-enviro-friendly end-of-life disposal of construction and demolition waste
Reading result:  161
Title:  Identifying potential opportunities of building information modeling for construction and demolition waste management and minimization
Reading result:  162
Title:  BIM-based Last Planner System tool for improving construction project management
Reading result:  163
Title:  A BIM-a

Title:  Building information modeling for sustainable design and LEED® rating analysis
Reading result:  197
Title:  Enhancing environmental sustainability over building life cycles through green BIM: A review
Reading result:  198
Title:  Mapping actual thermal properties to building elements in gbXML-based BIM for reliable building energy performance modeling
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Green%20Buildings%22
pageResults:  10
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Green%20Buildings%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  199
Title:  Building Information Modeling (BIM) for green buildings: A critic

...

Title:  Efficiency analysis of Set-based Design with structural building information modeling (S-BIM) on high-rise building structures
Reading result:  539
Title:  BIM-based estimation of vertical transportation demands during the construction of high-rise buildings
Reading result:  540
Title:  BIM-based collaborative design and socio-technical analytics of green buildings
Reading result:  541
Title:  A dedicated collaboration platform for Integrated Project Delivery
Reading result:  542
Title:  Formalized control logic fault definition with ontological reasoning for air handling units
Reading result:  543
Title:  Integrating building and urban semantics to empower smart water solutions
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B%20%22Thermal%20comfort%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20information%20model%22%20%2B%20%22Thermal%20comfort%22&show=25&publicat

base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Energy%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Water%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Energy%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  555
Title:  Bibliometric analysis and review of Building Information Modelling literature published between 2005 and 2015
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Water%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXE

base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Water%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  htt

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Energy%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Water%22&show=25&publicationTitles=271427&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query: 

In [11]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,A performance data integrated BIM framework fo...,Abstract,Data-driven green building design in BIM is ch...,1. Introduction,Energy shortages and the related carbon emissi...,2. P-BIM framework,Facing the existing problem of data integratio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
1,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Building Information Modeling (BIM) for green ...,Abstract,Although a large number of studies on Building...,1. Introduction,The building and construction industry has bee...,2. Research methodology,Connecting BIM and green buildings is the conc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
2,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,A BIM-WMS integrated decision support tool for...,Abstract,Selection of construction material sources is ...,1. Introduction,It is estimated that the cost of materials acc...,2. Literature review,As transportation is crucial in material plann...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
3,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Integrating web map service and building infor...,Abstract,"In green buildings design and construction, ev...",1. Introduction,It is widely perceived that the construction i...,2. Studies about current BIM applications in g...,Kriegel and Nies [4] found that BIM can help d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
4,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Green building assessment tool (GBAT) for inte...,Abstract,The benefits of BIM such as effective decision...,1. Introduction,The necessity of integrated data approach has ...,2. Related work,"Green BIM, as an emerging trend, has been incr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Informetric analysis and review of literature ...,Abstract,The construction industry and researchers have...,1. Introduction,Metropolitan areas are considered primary cont...,2. Methodology,The authors performed a systematic analysis of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
553,Digital%20construction,Sustainability,https://www.sciencedirect.com//science/article...,3D printing system for earth-based constructio...,Abstract,Despite the dramatic development in digital ma...,1. Introduction,An increasing amount of research on implementi...,2. Methods and material,"In cob constructon, printing material properti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
554,Digital%20construction,Sustainability,https://www.sciencedirect.com//science/article...,Bibliometric analysis and review of Building I...,Abstract,The use of Building Information Modelling (BIM...,1. Introduction,The construction industry currently generates ...,2. Methodology,Our paper analyses and classifies the research...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427
555,Digital%20construction,Energy,https://www.sciencedirect.com//science/article...,Bibliometric analysis and review of Building I...,Abstract,The use of Building Information Modelling (BIM...,1. Introduction,The construction industry currently generates ...,2. Methodology,Our paper analyses and classifies the research...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271427


In [12]:
df1 = df
df['journalName'] = str(idNo)
fileSave = 'kgRawData/SD/sd-kg'+idNo+'.csv'
df.to_csv(fileSave)

In [1]:
idNo = '271427'

import pandas as pd
# filePath = 
data = pd.read_csv('kgRawData/SD/sd-kg' + idNo + '.csv')
data['presence'] = False

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

uniqueValidEntries = list(set(validEntries))

uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''
    
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]    
    

uniqueDf.to_csv('kgRawData/SD/cleanSd-kg'+ idNo + '.csv')

print(len(uniqueDf))

<ipython-input-1-91996d7d20c3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


115


In [2]:
uniqueDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,Building%20information%20model,Energy,https://www.sciencedirect.com//science/article...,A rule-based system to automatically validate ...,Abstract,To facilitate BIM-to-Building Energy Modeling ...,1. Introduction,"In recent decades, numerous Building Informati...",2. Related work,With the increasing use of IFC as a standard f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Integrating web map service and building infor...,Abstract,"In green buildings design and construction, ev...",1. Introduction,It is widely perceived that the construction i...,2. Studies about current BIM applications in g...,Kriegel and Nies [4] found that BIM can help d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Review of BIM's application in energy simulati...,Abstract,Building Information Modeling (BIM) initially ...,1. Introduction,In order to study the shortcomings in BIM-to-B...,2. Types of BIM file schemas and their properties,BIM file schemas are the second link of the ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Building information modeling for sustainable ...,Abstract,"Today, there is a high level of demand for sus...",1. Introduction,Recent studies indicate that the demand for su...,"2. Purpose of the study, scope and methodology",The purpose of this study is to demonstrate th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,Integrating web map service and building infor...,Abstract,"In green buildings design and construction, ev...",1. Introduction,It is widely perceived that the construction i...,2. Studies about current BIM applications in g...,Kriegel and Nies [4] found that BIM can help d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Building%20information%20model,Energy,https://www.sciencedirect.com//science/article...,Development of openBIM-based energy analysis s...,Abstract,Energy performance assessments (EPAs) are wide...,1. Introduction,The construction sector accounts for 23% of ov...,2. EPA in the BIM environment,An EPA is a process in which multiple discipli...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Building%20information%20model,Energy,https://www.sciencedirect.com//science/article...,A rule-based system to automatically validate ...,Abstract,To facilitate BIM-to-Building Energy Modeling ...,1. Introduction,"In recent decades, numerous Building Informati...",2. Related work,With the increasing use of IFC as a standard f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,Building%20information%20model,Energy,https://www.sciencedirect.com//science/article...,A BIM-enabled information infrastructure for b...,Abstract,Although energy-efficient building technologie...,1. Introduction and motivation,The commercial and residential building sector...,2. Development of BIM I2,The proposed overall BIM enabled information i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,Building%20information%20model,Energy,https://www.sciencedirect.com//science/article...,Review of BIM's application in energy simulati...,Abstract,Building Information Modeling (BIM) initially ...,1. Introduction,In order to study the shortcomings in BIM-to-B...,2. Types of BIM file schemas and their properties,BIM file schemas are the second link of the ch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
